In [4]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors3D
from rdkit.Chem import rdChemReactions
from rdkit.Chem import Descriptors
from rdkit.Chem import Lipinski
from rdkit.Chem import rdChemReactions
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Crippen
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import SaltRemover
import pandas 
import csv
import freesasa
import numpy as np
import itertools

In [44]:
df = pandas.read_csv('FKBP-CIP-DEL-BB structures.csv')
dfe = pandas.read_csv('lib300_enumerated.csv')
dfno = pandas.read_csv('ToAppendNO2.csv')
dfnh = pandas.read_csv('ToAppendNH2.csv')
df_index_app = pandas.read_csv('append_index.csv')

lib_id = 300

In [56]:
dir_name = '/Users/ztan/Documents/FKBP CIP-DEL/Library Production/enumeration/'

In [6]:
rxn_amide = rdChemReactions.ReactionFromSmarts('[#7;A;X3;H1,H2!$(NS);!$(NC=O):1][2H]>>[3H]COC1=CC=C(CC[C@@H](NC(=O)C2CCCCN2C(=O)C(=O)C(C)(C)CC)C([N:1])=O)C=C1')
rxn_amine2 = rdChemReactions.ReactionFromSmarts('[#7;!$(NC=O):1][2H]>>[3H]COC1=CC=C(CC[C@@H](NC(=O)C2CCCCN2C(=O)C(=O)C(C)(C)CC)C[N:1])C=C1')
rxn_triazine = rdChemReactions.ReactionFromSmarts('[N:1]S>>[N:1]C1=NC(I)=NC([At])=N1')
rxn_triazine2 = rdChemReactions.ReactionFromSmarts('[#7;A;X3;H1,H2!$(NS);!$(NC=O):1]>>[N:1]C1=NC(I)=NC([At])=N1')
rxn_sub1 = rdChemReactions.ReactionFromSmarts('[#6:2]I.[#7;A;X3;H1,H2!$(NS);!$(NC=O):1]>>[#6:2]-[#7:1]')
rxn_sub2 = rdChemReactions.ReactionFromSmarts('[#6:2][At].[#7;A;X3;H1,H2!$(NS);!$(NC=O):1]>>[#6:2]-[#7:1]')
rxn_sub0 = rdChemReactions.ReactionFromSmarts('[#6:1][At]>>[#6:1][Cl]')
remover = Chem.SaltRemover.SaltRemover(defnData="[Cl,Br]")

structures = []
libid = [300]
counter = 0

structures = []
emw = []
fsp3 = []
nrb = []
slogp = []


In [7]:
#final with weird exceptions in Cy1 and Cy3
structures = []
counter = 0

for lib in libid:
    cy1 = df[(df['master_lib_id'] == lib) & (df['cy_no'] == 1)]
    cy2 = df[(df['master_lib_id'] == lib) & (df['cy_no'] == 2)]
    cy3 = df[(df['master_lib_id'] == lib) & (df['cy_no'] == 3)]
    
    for i in range(44):
        r1 = cy1.iloc[i]['bb_smiles']
        if rxn_amide.RunReactants((Chem.MolFromSmiles(r1),)) == ():
            #for shortest connector
            
            products = rxn_amine2.RunReactants((Chem.MolFromSmiles(r1),))
            p1 = Chem.MolToSmiles(products[0][0], isomericSmiles = True)    
            products = rxn_triazine2.RunReactants((Chem.MolFromSmiles(p1),))
            p1 = Chem.MolToSmiles(products[0][0], isomericSmiles = True)
        else:
            products = rxn_amide.RunReactants((Chem.MolFromSmiles(r1),))
            p1 = Chem.MolToSmiles(products[0][0], isomericSmiles = True)
            products = rxn_triazine.RunReactants((Chem.MolFromSmiles(p1),))
            p1 = Chem.MolToSmiles(products[0][0], isomericSmiles = True)
        for j in range(290):
            r2 = cy2.iloc[j]['bb_smiles']
            r2 = Chem.MolToSmiles(remover.StripMol(Chem.MolFromSmiles(r2)))
            products = rxn_sub1.RunReactants((Chem.MolFromSmiles(p1),Chem.MolFromSmiles(r2)))
            p2 = Chem.MolToSmiles(products[0][0], isomericSmiles = True)
            for k in range(248):
                r3 = cy3.iloc[k]['bb_smiles']
                r3 = Chem.MolToSmiles(remover.StripMol(Chem.MolFromSmiles(r3)))
                if rxn_sub2.RunReactants((Chem.MolFromSmiles(p2),Chem.MolFromSmiles(r3))) == ():
                    #for Cl
                    products = rxn_sub0.RunReactants((Chem.MolFromSmiles(p2),))
                    p3 = Chem.MolToSmiles(products[0][0], isomericSmiles = True)    

                else:
                    products = rxn_sub2.RunReactants((Chem.MolFromSmiles(p2),Chem.MolFromSmiles(r3)))
                    p3 = Chem.MolToSmiles(products[0][0], isomericSmiles = True)

                struct_out = Chem.MolToSmiles(products[0][0], isomericSmiles = True)
                structures.append(Chem.MolToSmiles(products[0][0], isomericSmiles = True))
                
                cpd_out = Chem.MolFromSmiles(struct_out)
                
                try:
                    emw.append(Descriptors.ExactMolWt(cpd_out))
                except:
                    emw.append(np.nan)
                try:
                    fsp3.append(Lipinski.FractionCSP3(cpd_out))
                except:
                    fsp3.append(np.nan)
                try:
                    nrb.append(Lipinski.NumRotatableBonds(cpd_out))
                except:
                    nrb.append(np.nan)
                try:
                    slogp.append(Crippen.MolLogP(cpd_out))
                except:
                    slogp.append(np.nan)
                    
        if counter == 43:
                structures.append(dfnh.loc[i,'structures'])
                emw.append(Descriptors.ExactMolWt(Chem.MolFromSmiles(dfnh.loc[i,'structures'])))
                fsp3.append(Lipinski.FractionCSP3(Chem.MolFromSmiles(dfnh.loc[i,'structures'])))
                nrb.append(Lipinski.NumRotatableBonds(Chem.MolFromSmiles(dfnh.loc[i,'structures'])))
                slogp.append(Crippen.MolLogP(Chem.MolFromSmiles(dfnh.loc[i,'structures'])))
        else:
                structures.append(dfno.loc[i,'structures'])
                emw.append(Descriptors.ExactMolWt(Chem.MolFromSmiles(dfno.loc[i,'structures'])))
                fsp3.append(Lipinski.FractionCSP3(Chem.MolFromSmiles(dfno.loc[i,'structures'])))
                nrb.append(Lipinski.NumRotatableBonds(Chem.MolFromSmiles(dfno.loc[i,'structures'])))
                slogp.append(Crippen.MolLogP(Chem.MolFromSmiles(dfno.loc[i,'structures'])))
                
                structures.append(dfnh.loc[i,'structures'])
                emw.append(Descriptors.ExactMolWt(Chem.MolFromSmiles(dfnh.loc[i,'structures'])))
                fsp3.append(Lipinski.FractionCSP3(Chem.MolFromSmiles(dfnh.loc[i,'structures'])))
                nrb.append(Lipinski.NumRotatableBonds(Chem.MolFromSmiles(dfnh.loc[i,'structures'])))
                slogp.append(Crippen.MolLogP(Chem.MolFromSmiles(dfnh.loc[i,'structures'])))
                
        counter += 1 
  
structures.append('C[C@@H]1CC[C@H]2C[C@H](OC)/C(C)=C/C=C/C=C/[C@@H](C)C[C@@H](C)C([C@H](OC)[C@H](O)/C(C)=C/[C@@H](C)C(C[C@@H]([C@@H](C[C@@H]3CC[C@@H](OC(CCC(O)=O)=O)[C@H](OC)C3)C)OC([C@@H]4CCCCN4C(C([C@]1(O)O2)=O)=O)=O)=O)=O')
rap = Chem.MolFromSmiles('C[C@@H]1CC[C@H]2C[C@H](OC)/C(C)=C/C=C/C=C/[C@@H](C)C[C@@H](C)C([C@H](OC)[C@H](O)/C(C)=C/[C@@H](C)C(C[C@@H]([C@@H](C[C@@H]3CC[C@@H](OC(CCC(O)=O)=O)[C@H](OC)C3)C)OC([C@@H]4CCCCN4C(C([C@]1(O)O2)=O)=O)=O)=O)=O')
fsp3.append(Lipinski.FractionCSP3(rap))
nrb.append(Lipinski.NumRotatableBonds(rap))
slogp.append(Crippen.MolLogP((rap)))
emw.append(Descriptors.ExactMolWt(rap))

dfe['structures'] = structures
dfe.to_csv('lib300_enumerated.csv', index = False)

lib_id = 300
pandas.DataFrame({'structures': structures}).to_csv(dir_name + 'meta/lib/enum_struct/lib' + str(lib_id).zfill(3) + '.csv', index = False)
pandas.DataFrame({'value': emw}).to_csv(dir_name + 'meta/lib/enum_prop/lib' + str(lib_id).zfill(3) + '_emw.csv', index = False)
pandas.DataFrame({'value': fsp3}).to_csv(dir_name + 'meta/lib/enum_prop/lib' + str(lib_id).zfill(3) + '_fsp3.csv', index = False)
pandas.DataFrame({'value': nrb}).to_csv(dir_name + 'meta/lib/enum_prop/lib' + str(lib_id).zfill(3) + '_nrb.csv', index = False)
pandas.DataFrame({'value': slogp}).to_csv(dir_name + 'meta/lib/enum_prop/lib' + str(lib_id).zfill(3) + '_slogp.csv', index = False)


len(structures)
len(fsp3)
len(emw)
len(slogp)
len(nrb)

KeyboardInterrupt: 

In [8]:
len(structures)

3164568

In [42]:
r1=cy1.iloc[4]['bb_smiles']
rxn_amide.RunReactants((Chem.MolFromSmiles(r1),))
rxn_amine2.RunReactants((Chem.MolFromSmiles(r1),))

((<rdkit.Chem.rdchem.Mol at 0x7faf2a287e70>,),)

In [59]:
# Generate meta cols
agg_meta = pd.read_csv(dir_name + 'meta/an/agg_meta.csv')

lib = 300
cy1 = df[(df['master_lib_id'] == lib) & (df['cy_no'] == 1)]
cy2 = df[(df['master_lib_id'] == lib) & (df['cy_no'] == 2)]
cy3 = df[(df['master_lib_id'] == lib) & (df['cy_no'] == 3)]

cy1.drop(cy1.tail(1).index,inplace=True)
cy2.drop(cy2.tail(2).index,inplace=True)
cy3.drop(cy3.tail(3).index,inplace=True)

tags = [cy1['cycle_id'], cy2['cycle_id'], cy3['cycle_id']]



all_combinations = list(itertools.product(*tags))
enum_df = pandas.concat([
    pandas.DataFrame({'lib_id': [lib_id for x in all_combinations]}),
    pandas.DataFrame(all_combinations, columns=['cycle1', 'cycle2', 'cycle3'])
], axis = 1)

frames = [enum_df,df_index_app]
final_df = pandas.concat(frames)

final_df = final_df.sort_values(by=['cycle1', 'cycle2','cycle3'])


for i in range(len(agg_meta)):
    agg_id = agg_meta.iloc[i]['agg_id']
    col_names = ['cycle' + x for x in agg_meta.iloc[i]['cyc'].split(', ')]
    col_names.insert(0, 'lib_id')
    
    enum_out = final_df[col_names].drop_duplicates()
    enum_out.to_csv(dir_name + 'meta/lib/meta_cols/lib' + str(lib_id).zfill(3) + '_' +
                    'agg' + str(agg_id).zfill(2) + '_meta_cols.csv', index = False)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [58]:
final_df

,lib_id,cycle1,cycle2,cycle3
0,300,1,1,1
1,300,1,1,2
2,300,1,1,3
3,300,1,1,4
4,300,1,1,5
...,...,...,...,...
83,300,41,291,250
84,300,42,291,250
85,300,43,291,250
86,300,44,291,250


In [54]:
final_df.sort_values(by=['cycle1', 'cycle2','cycle3'])

,lib_id,cycle1,cycle2,cycle3
0,300,1,1,1
1,300,1,1,2
2,300,1,1,3
3,300,1,1,4
4,300,1,1,5
...,...,...,...,...
3164477,300,44,290,246
3164478,300,44,290,247
3164479,300,44,290,248
86,300,44,291,250


In [ ]:
pandas.